In [33]:
# Jeff Powell
# 11/29/23
# Notebook explores possible uses for the itertools module of the python standard
# library, and its extensions with the more-itertools package
# CSC295

In [34]:
# Imperative programming is perhaps the most commonly taught style of programming. It requires users
# To handle explicit storage of variables, changing of value, and often includes indexing into collections.
# Despite this, lots of times, imperative programming can become complex and the implementation 
# of code can begin to hide the intention of the programmer. Functional programming is a different
# style of coding that can allow programmers to express intent more directly and concisely, while
# readers can have a clearer indication of what the programmer was attempting to accomplish.
# Functional programming focuses on using functions that do not mutate their parameters as well as
# using a construct called iterators to act on data structures rather than keeping track of indicies everywhere.
# Itertools and more-itertools express ways for programmers to act on a collection and extract information
# from it. More-itertools was first released in April, 2012. It is currently on version 10.1 which was
# released in August, 2023. This package would pair very well with the functools package to enable more
# functional programming constructs.

In [35]:
import sys
import itertools
import more_itertools as mit

In [36]:
# This is a contrived example that takes in two integers. If bottom is greater than top, zero is
# returned. Otherwise, the function inclusively iterates over the range of numbers between bottom
# and top, and accumulates the even ones. It is not terribly complicated, but as the conditions
# grow more complex, it will get longer and longer
def calculate(bottom: int, top: int):
    if top > bottom:
        sum = 0
        for i in range(bottom, top + 1):
            if i % 2 == 0:
                sum += i
        return sum

    else:
        return 0


# This function preforms the same operation as the previous function, but in a functional manner
# with itertools.
def calculate_iter(bottom: int, top: int):
    if top > bottom:
        # This function is a comparator that returns true if a number is even. It could easily
        # and clearly be modified if more restrictions were placed on the numbers that should be
        # accumulated.
        keyfunc = lambda x: x % 2
        # The map_reduce function takes in an iterable as the first parameter, the filtering function
        # as the 'keyfunc' parameter, and reducefunc reduces the result. Map_reduce returns a dictionary
        # with keys of possible results to the keyfunc parameter, and values in this case of the summed
        # even numbers in the range since we specified sum as the reducefunc. The dictionary also
        # contains the odd numbers in the range summed as the value to the other key in the dict.
        filtered = mit.map_reduce(range(bottom, top + 1), keyfunc, reducefunc=sum)
        # We return the value at key 0 since we want even numbers
        return filtered[0]
    else:
        return 0

print(calculate(4, 8))
print(calculate_iter(4, 8))
print()
print(calculate(8, 4))
print(calculate_iter(8, 4))

18
18

0
0


In [37]:
# Another problem that can easily be solved with more-itertools is problems that require iterating 
# through "windows" of a collection. For example, if I have a list [1, 2, 3, 4, 5] and I want windows
# of three elements each, the iterator would sequentially return [1, 2, 3], [2, 3, 4], and [3, 4, 5].
# In this example the functions will calculate the highest sum of four consecutive elements.

data = [2, 5, 7, 1, 9, 4, 12, 10, 5, 2, 25]
WINDOW_SIZE = 4

def max_sum(data: list[int]):
    # We have to have explicit tracking variables for the current max sum and window found
    max = -sys.maxsize
    max_window = []
    # The code for creating these windows is very prone to off by one errors and mental load by readers
    for i in range(len(data) - (WINDOW_SIZE - 1)):
        window = []
        for j in range(WINDOW_SIZE):
            window.append(data[i + j])

        sum = 0
        for element in window:
            sum += element
        if sum > max:
            max_window = window
            max = sum
    return (max, max_window)


def max_sum_functional(data: list[int]):
    # The windowed function simply requires an argument of window size. Furthermore, if the
    # spacing between windows should change, the windowed method can take an argument to modify
    # that value as well as opposed to the imperative code which would have to be largely rewritten.
    windows = mit.windowed(data, WINDOW_SIZE)
    # The max function returns the largest element found, with the key value specififying we want the
    # largest element by nature of the summed window.
    w = max(windows, key=sum)
    return (sum(w), w)

# Interestingly, the windowed method returns a tuple instead of a list as was created in the imperative        
# code. This works to ensure the user cannot change the data of the window, and mistake changing
# the window for changing the underlying data or something similar.
print(max_sum(data))
print(max_sum_functional(data))

(42, [10, 5, 2, 25])
(42, (10, 5, 2, 25))


In [38]:
# A third example could be removing the names in a list that start with a certain letter, and then returning
# names that match that criteria with their first letter upper-case

NAMES = ["james", "robert", "john", "michael", "david", "joshua"]
TARGET_LETTER = 'J'

def filter_names(data: list[str], target_letter: str):
    new_list = []
    for name in data:
        if name[0].upper() == target_letter:
            changed_name = name[0].upper() + name[1:]
            new_list.append(changed_name)
    return new_list


def filter_names_functional(data: list[str], target_letter: str):
    # We are interested in letters where the first letter is J. 
    # Calling upper ensures it does not matter what case the user inputs
    keyfunc = lambda x: x[0].upper() == target_letter
    # Transform the resulting names by making the first letter uppercase
    # Another benefit is individuals familiar with this coding style know
    # exactly where to look for how the data is being transformed, as it
    # is performed in one consistent place.
    valuefunc = lambda x: x[0].upper() + x[1:]
    # Map_reduce is a phenomenally useful function because of just how often in programming the act
    # of taking a collection, changing the data type, and then modifying or summarizing the output
    filtered_names = mit.map_reduce(data, keyfunc, valuefunc)
    return filtered_names[True]


print(filter_names(NAMES, TARGET_LETTER))
print(filter_names_functional(NAMES, TARGET_LETTER))


['James', 'John', 'Joshua']
['James', 'John', 'Joshua']


In [39]:
# A final example involves interleaving data. I know that this is important in the field of neural networks,
# as if too much data in a row comes from one source of data it can cause the neural network to overfit
# to one particular aspect of its feature set. A worry doing this imperatively is ensuring that if one list
# is shorter than the other, it is important to pull the rest of the elements from the second list into the final 
# collection.

LEFT = [1, 2, 4, 8]
RIGHT = [0, 3, 6, 9, 12]

def interleave(left: list[int], right: list[int]):
    left_len = len(left)
    right_len = len(right)
    min_len = min(left_len, right_len)
    result = []
    for i in tuple(zip(left, right)):
        # The zip function can be used to tie together two lists into an object that yields one element from each
        # collection at a time until one is exhausted
        result.append(i[0])
        result.append(i[1])
    # The "until one collection is exhausted" behavior of zipping means we have to use this next chunk of code to 
    # append the rest of the remaining collection to the result. And this would only get more complex as there are
    # more collections, unlike more-itertool's interleave_longest function which simply takes collections as its 
    # parameters and returns the single list you want
    if min_len == left_len:
        for i in right[min_len:]:
            result.append(i)
    else:
        for i in left[min_len:]:
            result.append(i)
    return result

def interleave_functional(left: list[int], right: list[int]):
    return list(mit.interleave_longest(left, right))

print(interleave(LEFT, RIGHT))
print(interleave_functional(LEFT, RIGHT))

[1, 0, 2, 3, 4, 6, 8, 9, 12]
[1, 0, 2, 3, 4, 6, 8, 9, 12]


In [40]:
# I found the documentation for this package very good. It has very usable code examples with good
# explanations for how the methods work and what each parameter for the functions does. Overall,
# this is a very generally useful package. Many projects will not find much use for it as they will
# find what little data transformation they need in other libraries or modules, but for projects that
# are implementing lots of logic in native python, this can be a very useful tool. The main drawback
# is that its technically not really necessary. Most of the time, imperatively written code will do
# just fine, and has the benefit of allowing a wider audience to read it. It takes a different style
# of thinking to read and write functional code compared to imperatively written code, which many people
# shouldn't be expected to have experience with.

In [41]:
# Web Resources
# - https://more-itertools.readthedocs.io/en/stable/api.html?highlight=distinct_permutation#augmenting
# - https://docs.python.org/3/howto/functional.html
# - https://realpython.com/python-itertools/